SDH subtitles from https://www.opensubtitles.org/en/ssearch/sublanguageid-ger/idmovie-578221

In [24]:
import re
import pandas as pd
import numpy as np
import sys
import os
import pysrt #for subs analysis
from datetime import date, datetime, timedelta, time

import matplotlib
from matplotlib import style
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (16.0, 9.0)
style.use('fivethirtyeight')

The first dataset is for vocabulary size calculation

In [87]:
# function that returns a dataframe for a given .srt file with text from all the lines
def text_from_srt(path):
    path = path
    pattern = r'\[(.*?)\]'
    lines=[]
    # loading the subs
    subs = pysrt.open(path)
    #converting to dataframe by combining all lines per episode into one text (without [...] sdh comments | music)
    for line in subs:
        text = line.text_without_tags.replace('\n',' ')
        # not interested in lyrics and opening text
        if '♪' not in text and 'NETFLIX' not in text:
            # checking whether it contains sdh comments and removing them
            text = re.sub(pattern,'',text)
            lines.append(text)
        
    # concatenating all the values from list to create a text
    clean_text = ' '.join(lines)
    return clean_text

In [112]:
directory = 'SDH/'
# all .srt files are named like this: S01E01.srt 

data = pd.DataFrame(columns =["episode_text","episode","season","season00episode00"])
i=0

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and '.srt' in filename:
        print(f)
        i+=1
        text = text_from_srt(f)
        # appending to a dataframe
        data = data.append({"episode_text":text,"episode":filename[-6:-4],
        "season":filename[1:3],"season00episode00":filename[0:6]},
        ignore_index = True)

        print(filename[0:6],"done, text length: ",len(text))

data.head(5)

SDH/S03E02.srt
S03E02 done, text length:  11089
SDH/S01E07.srt
S01E07 done, text length:  11794
SDH/S01E06.srt
S01E06 done, text length:  11988
SDH/S03E03.srt
S03E03 done, text length:  11569
SDH/S03E01.srt
S03E01 done, text length:  12846
SDH/S01E04.srt
S01E04 done, text length:  11558
SDH/S01E10.srt
S01E10 done, text length:  14168
SDH/S01E05.srt
S01E05 done, text length:  12000
SDH/S03E04.srt
S03E04 done, text length:  12915
SDH/S01E01.srt
S01E01 done, text length:  14919
SDH/S03E05.srt
S03E05 done, text length:  11043
SDH/S03E07.srt
S03E07 done, text length:  12737
SDH/S01E02.srt
S01E02 done, text length:  7375
SDH/S01E03.srt
S01E03 done, text length:  8602
SDH/S03E06.srt
S03E06 done, text length:  12883
SDH/S02E08.srt
S02E08 done, text length:  13514
SDH/S02E01.srt
S02E01 done, text length:  11861
SDH/S02E03.srt
S02E03 done, text length:  16212
SDH/S02E02.srt
S02E02 done, text length:  13237
SDH/S02E06.srt
S02E06 done, text length:  12861
SDH/S02E07.srt
S02E07 done, text length:  

,episode_text,episode,season,season00episode00
0,- - Zieh dich um. -Martha? - Das kann nic...,02,03,S03E02
1,Ich erinnere mich. Ich erinnere mich an ...,07,01,S01E07
2,Das ist nicht mehr lustig! Katharina? Ulr...,06,01,S01E06
3,"""Von da an wusste ich, dass sich nichts ände...",03,03,S03E03
4,"Wenn wir wüssten, wie die Dinge enden... ......",01,03,S03E01


The second dataset is for SDH comments mapping — e.g. we're interested in how often something happens like the music starts playing. This is why we're saving intervals here.

In [76]:
# в 10 сабе 2 коммента, в 130 - 1, в 138 - 1 и есть доп текст
# print(subs[101].text_without_tags)

text = subs[138].text_without_tags.replace('\n',' ')
print(text)

pattern = r'\[(.*?)\]'

# sdh_comments = []
# sdh_interval_starts = []
matches = re.findall(pattern,text)
if len(matches)>0:
    # for match in re.findall(pattern,text):
    #     print(match)
    replacements = re.subn(pattern,'',text)
    print("clean string: "+replacements[0])
    print("num of replacements: ",replacements[1])    
    # sdh_comments.append(match)
    # sdh_interval_starts.append('time')
# sdh_len = len(sdh_comments)



# if not text.startswith('♪') and 'NETFLIX' not in text:
#     print('not opening or music')

[Mann] Euch geht das am Arsch vorbei! Niemand macht was!
clean string:  Euch geht das am Arsch vorbei! Niemand macht was!
num of replacements:  1


The third dataset can be created to save lines for intervals like 10 min or 30 min to see the tone dynamic or something. But this is something we'll do later (maybe).

Good example can be found here: https://mubaris.com/posts/movie-analysis/

In [ ]:
# #loading the subtitles
# path ='s01/SDH/S01E01.srt'
# subs = pysrt.open(path)
# print(len(subs))

# lines = []
# interval_starts = []
# for line in subs:
#     text = line.text_without_tags.replace('\n',' ')
#     # not interested in lyrics and opening text
#     if '♪' not in text and 'NETFLIX' not in text:
#         lines.append(text)
#         interval_starts.append(line.start.to_time())

# sub_len = len(lines)
# print(sub_len)

# data = pd.DataFrame()
# data["lines"] = lines
# data["interval_starts"]=interval_starts

Some silly testing

In [17]:
# start = time(0, 0, 0)
# end = subs[-1].end.to_time()

datetime.time(0, 48, 18, 916000)

In [ ]:
# stop_words = stopwords.words('german')
# including Winden as it's just a city name + characters names
# stop_words.extend(['Winden','Jonas','Adam','Eva','Hanno','Noah','The Unknown',\
#     'Mikkel','Ines','Michael','Magnus','Bartosz','Egon','Mads','Silja','Agnes',\
#     'Ulrich', 'Katharina','Hannah','Martha','Jana','Tronte',\
#     'Charlotte','Regina','Franziska','Peter','Helge','Aleksander','Boris','Claudia','Elisabeth','Yasin',\
#     'Albers','Kahnwald','Nielsen','Doppler','Tiedemann','Niewald','Tannhaus','Tauber','Wöller','Krüger','Obendorf'])